# Clone Simplified Data
# Organize Data Into Directories
# Convert Images
# Convert Labels

In [ ]:
import os
import shutil

In [ ]:
src_dir = f"[path]"
dest_dir = f"[path]"

## Simple copying files from one folder to other.

##### This section picks simplified directories which have image and label and make a clone of this directory

In [ ]:
def copy_files(src, dst):
    
    for root, dirs, files in os.walk(src):
        
        dest_dir = root.replace(src, dst, 1)
        
        
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
            
            
        for file in files:
            parent_dir = root.split("\\")[-1]
            src_file = os.path.join(root, file)
            dest_file = os.path.join(dest_dir, file)
            shutil.copy2(src_file, dest_file)
            print(parent_dir)
            print(f'Copied: {src_file} to {dest_file}', end="\n\n")

In [ ]:

# copy_files(src_dir, dest_dir)

## Organizing Folders

##### This section picst simplified directory and create a new directory with Organized data

In [ ]:
import os
import shutil

def organize_files(src_dir, dst_dir):
    
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            
            if not file.lower().endswith('.nii.gz'):
                
                base_name = os.path.splitext(file)[0]


                relative_path = os.path.relpath(root, src_dir)
                new_folder_path = os.path.join(dst_dir, relative_path, base_name)


                if not os.path.exists(new_folder_path):
                    os.makedirs(new_folder_path)


                src_file = os.path.join(root, file)
                dest_file = os.path.join(new_folder_path, file)
                shutil.copy2(src_file, dest_file)
                print(f'\nMoved: {src_file} to {dest_file}')


                label_file = f"{base_name}.nii.gz"
                label_file_path = os.path.join(root, label_file)
                if os.path.exists(label_file_path):
                    dest_label_file = os.path.join(new_folder_path, label_file)
                    shutil.copy2(label_file_path, dest_label_file)
                    print(f'Moved label file: {label_file_path} to {dest_label_file}')
                else:
                    print(f'Label file not found: {label_file_path}')

organize_files(src_dir, dest_dir)


## Complete Conversion and Saving Code.

In [ ]:
import os
import subprocess
import shutil
import nibabel as nib
import numpy as np
import re

def reorient_label_to_match(image_path, label_path, output_label_path):
    """
    Reorients the label NIfTI file to match the orientation of the reference image.

    Parameters:
        image_path (str): Path to the reference NIfTI file (e.g., converted DICOM).
        label_path (str): Path to the label NIfTI file to be reoriented.
        output_label_path (str): Path to save the reoriented label file.
    """
    
    image_nii = nib.load(image_path)
    label_nii = nib.load(label_path)


    image_orientation = nib.aff2axcodes(image_nii.affine)
    label_orientation = nib.aff2axcodes(label_nii.affine)

    print("Image Orientation:", image_orientation)
    print("Label Orientation:", label_orientation)


    ornt_transform = nib.orientations.ornt_transform(
        nib.orientations.axcodes2ornt(label_orientation),
        nib.orientations.axcodes2ornt(image_orientation)
    )


    label_data = label_nii.get_fdata()
    label_affine = label_nii.affine

    reoriented_label_data = nib.orientations.apply_orientation(label_data, ornt_transform)
    reoriented_label_affine = label_affine @ nib.orientations.inv_ornt_aff(ornt_transform, label_data.shape)

    reoriented_label_nii = nib.Nifti1Image(reoriented_label_data, reoriented_label_affine)
    nib.save(reoriented_label_nii, output_label_path)

    print(f"Reoriented label saved to: {output_label_path}", end="\n\n")

    
def convert_dicom_to_nifti(dicom_dir, output_dir, output_filename):
    command = f'dcm2niix -o "{output_dir}" -f "{output_filename}" -z y "{dicom_dir}"'
    print(f"Running Command: {command}")
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    print("STDOUT:", result.stdout)
    print("STDERR:", result.stderr)


    
def get_nifti_filename(directory):
    """
    Read file name that is saved recently
    """
    
    nifti_files = [f for f in os.listdir(directory) if f.endswith('.nii.gz')]

    if not nifti_files:
        print("No .nii.gz file found in the directory.")
        return None
    
    nifti_file = nifti_files[0]
    print(f"Found NIfTI file: {nifti_file}")
    return nifti_file    
    


def process_folders(root_dir, output_base_dir):
    
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            
            if not file.lower().endswith('.nii.gz'):

                dicom_filename = file
                label_filename = f"{dicom_filename}.nii.gz"

                dicom_dir = os.path.join(root, dicom_filename)
                label_file = os.path.join(root, label_filename)

                relative_path = os.path.relpath(root, root_dir)
                output_dir = os.path.join(output_base_dir, relative_path)
                
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                parent_dir = root.split("\\")[-2]

                output_filename = dicom_filename
                output_filename = f"{dicom_filename}_{parent_dir}_%s_%t_%i_%z"
                convert_dicom_to_nifti(dicom_dir, output_dir, output_filename)
                converted_file_name = get_nifti_filename(output_dir)
                
                if os.path.exists(label_file):
                    dest_img_file = os.path.join(output_dir, converted_file_name)
                    dest_label_file = os.path.join(output_dir, label_filename)
                    reorient_label_to_match(dest_img_file, label_file, dest_label_file)
                    print(f"Re-oriented label is saved to {dest_label_file}")
                else:
                    print(f"Label file not found: {label_file}")

        
root_dir = f"[path]"
output_base_dir = f"[path]"

process_folders(root_dir, output_base_dir)
